In [155]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score,recall_score, confusion_matrix, classification_report,accuracy_score, f1_score
import numpy as np
import pandas as pd
import random
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve , auc
from sklearn.metrics.cluster import fowlkes_mallows_score as gmean
import statistics as stat

In [156]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [157]:
df=pd.read_csv('/content/drive/MyDrive/page-blocks-1-3_vs_4.csv')
df

,Height,Lenght,Area,Eccen,P_black,P_and,Mean_tr,Blackpix,Blackand,Wb_trans,Class
0,3,113,339,37.667,0.410,0.472,13.90,139,160,10,negative
1,3,59,177,19.667,0.475,0.475,84.00,84,84,1,negative
2,2,20,40,10.000,0.675,0.675,27.00,27,27,1,negative
3,2,101,202,50.500,0.540,0.743,10.90,109,150,10,negative
4,4,194,776,48.500,0.263,0.273,34.00,204,212,6,negative
...,...,...,...,...,...,...,...,...,...,...,...
467,65,50,3250,0.769,0.454,0.487,245.83,1475,1582,6,positive
468,63,79,4977,1.254,0.338,0.345,186.78,1681,1718,9,positive
469,178,140,24920,0.787,0.574,0.649,21.52,14292,16177,664,positive
470,166,472,78352,2.843,0.359,0.547,17.12,28093,42821,1641,positive


In [158]:
df["Class"].value_counts()
class_label=df['Class']
class_label

0      negative
1      negative
2      negative
3      negative
4      negative
         ...   
467    positive
468    positive
469    positive
470    positive
471    positive
Name: Class, Length: 472, dtype: object

In [159]:
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df.head()


,Height,Lenght,Area,Eccen,P_black,P_and,Mean_tr,Blackpix,Blackand,Wb_trans,Class
0,3,113,339,37.667,0.410,0.472,13.9,139,160,10,0
1,3,59,177,19.667,0.475,0.475,84.0,84,84,1,0
2,2,20,40,10.000,0.675,0.675,27.0,27,27,1,0
3,2,101,202,50.500,0.540,0.743,10.9,109,150,10,0
4,4,194,776,48.500,0.263,0.273,34.0,204,212,6,0


In [160]:
minor=df[df["Class"]==1]
major=df[df["Class"]==0]
minor=minor.drop(["Class"], axis=1)
minority=minor.to_numpy()
minor.shape
major.shape
diff=major.shape[0]-minor.shape[0]
r=diff
loop=diff/minor.shape[0]
if(diff%minor.shape[0]==0):
    iter=int(diff/minor.shape[0])
else:
  iter=int(diff/minor.shape[0])+1
minor.shape


(28, 10)

In [161]:
for id in range(r):
  if(diff>0):
    distances = np.linalg.norm(minority - minority[id], axis=1)  #calculating the distances to other instances
    k = 4
    nearest_neighbor_ids = distances.argsort()[:k] #finding 3 nearest neighbours n1,n2 and n3
    n1=minority[nearest_neighbor_ids[1]]
    n2=minority[nearest_neighbor_ids[2]]
    n3=minority[nearest_neighbor_ids[3]]
    r1=random.uniform(0,1)
    r2=random.uniform(0,1)
    w=random.uniform(0.5,1)
    velocity= w+r1*(n1-n3) + r2*(n1-n2)         #velocity vector
    new_minor=minority[id]+velocity             #position update
    d=minor.shape[1] 
    minB=minor.min().to_numpy()
    maxB=minor.max().to_numpy()
    for i in range(d):
      if(new_minor[i]<minB[i]):
        new_minor[i]=minB[i];
      if(new_minor[i]>maxB[i]):
        new_minor[i]=maxB[i];
    minority=np.append(minority,new_minor)
    minority=np.reshape(minority,(-1, minor.shape[1]))
    diff=diff-1

minority.shape

(444, 10)

In [162]:
df_minor = pd.DataFrame(minority, columns =['Height','Lenght','Area','Eccen','P_black','P_and','Mean_tr','Blackpix','Blackand','Wb_trans'])
df_minor

,Height,Lenght,Area,Eccen,P_black,P_and,Mean_tr,Blackpix,Blackand,Wb_trans
0,28.000000,31.000000,868.000000,1.107,0.492,0.745,7.490000,427.000000,647.000000,57.000000
1,28.000000,32.000000,896.000000,1.143,0.481,0.742,7.430000,431.000000,665.000000,58.000000
2,29.000000,31.000000,899.000000,1.069,0.463,0.726,6.930000,416.000000,653.000000,60.000000
3,28.000000,32.000000,896.000000,1.143,0.488,0.743,7.050000,437.000000,666.000000,62.000000
4,28.000000,32.000000,896.000000,1.143,0.462,0.722,7.020000,414.000000,647.000000,59.000000
...,...,...,...,...,...,...,...,...,...,...
439,187.000000,31.000000,46857.442831,4.144,0.899,0.991,43.004889,402.728415,46133.000000,2333.000000
440,187.000000,547.000000,87234.000000,4.144,0.899,0.991,245.830000,33017.000000,46133.000000,18.772159
441,54.047578,547.000000,20486.909583,4.144,0.899,0.991,245.830000,401.000000,639.000000,16.067310
442,34.393030,127.904492,872.207204,4.144,0.899,0.991,12.730434,7456.616664,3743.700521,12.393030


In [163]:
dup=[]
for i in range(df_minor.shape[0]):
  dup.append(1)
Class=np.array(dup)
Class=np.reshape(Class,(-1,1))
df_minor_class=pd.DataFrame(Class,columns=['Class'])
df_minor['Class']=df_minor_class
df_minor

,Height,Lenght,Area,Eccen,P_black,P_and,Mean_tr,Blackpix,Blackand,Wb_trans,Class
0,28.000000,31.000000,868.000000,1.107,0.492,0.745,7.490000,427.000000,647.000000,57.000000,1
1,28.000000,32.000000,896.000000,1.143,0.481,0.742,7.430000,431.000000,665.000000,58.000000,1
2,29.000000,31.000000,899.000000,1.069,0.463,0.726,6.930000,416.000000,653.000000,60.000000,1
3,28.000000,32.000000,896.000000,1.143,0.488,0.743,7.050000,437.000000,666.000000,62.000000,1
4,28.000000,32.000000,896.000000,1.143,0.462,0.722,7.020000,414.000000,647.000000,59.000000,1
...,...,...,...,...,...,...,...,...,...,...,...
439,187.000000,31.000000,46857.442831,4.144,0.899,0.991,43.004889,402.728415,46133.000000,2333.000000,1
440,187.000000,547.000000,87234.000000,4.144,0.899,0.991,245.830000,33017.000000,46133.000000,18.772159,1
441,54.047578,547.000000,20486.909583,4.144,0.899,0.991,245.830000,401.000000,639.000000,16.067310,1
442,34.393030,127.904492,872.207204,4.144,0.899,0.991,12.730434,7456.616664,3743.700521,12.393030,1


In [164]:
majority=major.to_numpy()
majority=np.reshape(majority,(-1,major.shape[1]))
majority
df_major = pd.DataFrame(majority, columns = ['Height','Lenght','Area','Eccen','P_black','P_and','Mean_tr','Blackpix','Blackand','Wb_trans','Class'])
df_major

,Height,Lenght,Area,Eccen,P_black,P_and,Mean_tr,Blackpix,Blackand,Wb_trans,Class
0,3.0,113.0,339.0,37.667,0.410,0.472,13.90,139.0,160.0,10.0,0.0
1,3.0,59.0,177.0,19.667,0.475,0.475,84.00,84.0,84.0,1.0,0.0
2,2.0,20.0,40.0,10.000,0.675,0.675,27.00,27.0,27.0,1.0,0.0
3,2.0,101.0,202.0,50.500,0.540,0.743,10.90,109.0,150.0,10.0,0.0
4,4.0,194.0,776.0,48.500,0.263,0.273,34.00,204.0,212.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
439,32.0,350.0,11200.0,10.938,0.142,0.402,3.96,1588.0,4497.0,401.0,0.0
440,31.0,105.0,3255.0,3.387,0.224,0.417,5.35,728.0,1358.0,136.0,0.0
441,28.0,61.0,1708.0,2.179,0.172,0.509,1.97,294.0,870.0,149.0,0.0
442,83.0,314.0,26062.0,3.783,0.298,0.671,8.75,7772.0,17482.0,888.0,0.0


In [165]:
balanced_df=pd.concat([df_minor,df_major],axis=0,ignore_index=True)
balanced_df

,Height,Lenght,Area,Eccen,P_black,P_and,Mean_tr,Blackpix,Blackand,Wb_trans,Class
0,28.0,31.0,868.0,1.107,0.492,0.745,7.49,427.0,647.0,57.0,1.0
1,28.0,32.0,896.0,1.143,0.481,0.742,7.43,431.0,665.0,58.0,1.0
2,29.0,31.0,899.0,1.069,0.463,0.726,6.93,416.0,653.0,60.0,1.0
3,28.0,32.0,896.0,1.143,0.488,0.743,7.05,437.0,666.0,62.0,1.0
4,28.0,32.0,896.0,1.143,0.462,0.722,7.02,414.0,647.0,59.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
883,32.0,350.0,11200.0,10.938,0.142,0.402,3.96,1588.0,4497.0,401.0,0.0
884,31.0,105.0,3255.0,3.387,0.224,0.417,5.35,728.0,1358.0,136.0,0.0
885,28.0,61.0,1708.0,2.179,0.172,0.509,1.97,294.0,870.0,149.0,0.0
886,83.0,314.0,26062.0,3.783,0.298,0.671,8.75,7772.0,17482.0,888.0,0.0


In [166]:

from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=5)
x_b=balanced_df.drop(['Class'],axis='columns')
y_b=balanced_df.Class




In [167]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
model_b_s=SVC()

model_b_k=KNeighborsClassifier(n_neighbors=3)


model_b_d=DecisionTreeClassifier(random_state=0)

In [168]:
#precision
precision_s=[]
f1_s=[]
recall_s=[]
accuracy_s=[]
Auc_s=[]
Gmean_s=[]

precision_k=[]
f1_k=[]
recall_k=[]
accuracy_k=[]
Auc_k=[]
Gmean_k=[]

precision_d=[]
f1_d=[]
recall_d=[]
accuracy_d=[]
Auc_d=[]
Gmean_d=[]
for train_index_b,test_index_b in kf.split(x_b,y_b):
  x_train_b,x_test_b=x_b.iloc[train_index_b],x_b.iloc[test_index_b]
  y_train_b,y_test_b=y_b.iloc[train_index_b],y_b.iloc[test_index_b]
  model_b_s.fit(x_train_b,y_train_b)
  prediction_b_s = model_b_s.predict(x_test_b)

  model_b_k.fit(x_train_b,y_train_b)
  prediction_b_k = model_b_k.predict(x_test_b)

  model_b_d.fit(x_train_b,y_train_b)
  prediction_b_d = model_b_d.predict(x_test_b)

  precision_s.append(precision_score(y_test_b,prediction_b_s))
  precision_k.append(precision_score(y_test_b,prediction_b_k))
  precision_d.append(precision_score(y_test_b,prediction_b_d))

  recall_s.append(recall_score(y_test_b,prediction_b_s))
  recall_k.append(recall_score(y_test_b,prediction_b_k))
  recall_d.append(recall_score(y_test_b,prediction_b_d))

  f1_s.append(f1_score(y_test_b,prediction_b_s))
  f1_k.append(f1_score(y_test_b,prediction_b_k))
  f1_d.append(f1_score(y_test_b,prediction_b_d))

  accuracy_s.append(accuracy_score(y_test_b,prediction_b_s))
  accuracy_k.append(accuracy_score(y_test_b,prediction_b_k))
  accuracy_d.append(accuracy_score(y_test_b,prediction_b_d))


  s_fpr_b,s_tpr_b,threshold_s_b=roc_curve(y_test_b,prediction_b_s)
  k_fpr_b,k_tpr_b,threshold_k_b=roc_curve(y_test_b,prediction_b_k)
  d_fpr_b,d_tpr_b,threshold_d_b=roc_curve(y_test_b,prediction_b_d)


  Auc_s.append(auc(s_fpr_b,s_tpr_b))
  Auc_k.append(auc(k_fpr_b,k_tpr_b))
  Auc_d.append(auc(d_fpr_b,d_tpr_b))


  Gmean_s.append(gmean(y_test_b,prediction_b_s))
  Gmean_k.append(gmean(y_test_b,prediction_b_k))
  Gmean_d.append(gmean(y_test_b,prediction_b_d))


print("-------SVM--------")
print("Gmean_SVM:",Gmean_s)
print("AUC_SVM:",Auc_s)
print("f1_SVM:",f1_s)
print("precision_SVM:",precision_s)
print("recall_SVM:",recall_s)

print("mean of Gmean_s:",stat.mean(Gmean_s))
print("mean of AUC_s:",stat.mean(Auc_s))
print("mean of f1_s:",stat.mean(f1_s))
print("mean of precision_s:",stat.mean(precision_s))
print("mean of recall_s:",stat.mean(recall_s))

print("standard deviation of Gmean_s:",stat.stdev(Gmean_s))
print("standard deviation of AUC_s:",stat.stdev(Auc_s))
print("standard deviation of f1_s:",stat.stdev(f1_s))
print("standard deviation of precision_s:",stat.stdev(precision_s))
print("standard deviation of recall_s:",stat.stdev(recall_s))

print("\n")
print("-------KNN--------")
print("Gmean_KNN:",Gmean_k)
print("AUC_KNN:",Auc_k)
print("f1_KNN:",f1_k)
print("precision_KNN:",precision_k)
print("recall_KNN:",recall_k)

print("mean of Gmean_k:",stat.mean(Gmean_k))
print("mean of AUC_k:",stat.mean(Auc_k))
print("mean of f1_k:",stat.mean(f1_k))
print("mean of precision_k:",stat.mean(precision_k))
print("mean of recall_k:",stat.mean(recall_k))

print("standard deviation of Gmean_k:",stat.stdev(Gmean_k))
print("standard deviation of AUC_k:",stat.stdev(Auc_k))
print("standard deviation of f1_k:",stat.stdev(f1_k))
print("standard deviation of precision_k:",stat.stdev(precision_k))
print("standard deviation of recall_k:",stat.stdev(recall_k))

print("\n")
print("-------DECISION TREE--------")
print("Gmean_DT:",Gmean_d)
print("AUC_DT:",Auc_d)
print("f1_DT:",f1_d)
print("precision_DT:",precision_d)
print("recall_DT:",recall_d)

print("mean of Gmean_d:",stat.mean(Gmean_d))
print("mean of AUC_d:",stat.mean(Auc_d))
print("mean of f1_d:",stat.mean(f1_d))
print("mean of precision_d:",stat.mean(precision_d))
print("mean of recall_d:",stat.mean(recall_d))

print("standard deviation of Gmean_d:",stat.stdev(Gmean_d))
print("standard deviation of AUC_d:",stat.stdev(Auc_d))
print("standard deviation of f1_d:",stat.stdev(f1_d))
print("standard deviation of precision_d:",stat.stdev(precision_d))
print("standard deviation of recall_d:",stat.stdev(recall_d))



-------SVM--------
Gmean_SVM: [0.6720304461993005, 0.7105981957003813, 0.6899866007980032, 0.6691208809001082, 0.6259579957879333]
AUC_SVM: [0.7640449438202247, 0.8089887640449438, 0.7977528089887641, 0.786006128702758, 0.7510214504596526]
f1_SVM: [0.6956521739130433, 0.7638888888888888, 0.7600000000000001, 0.7564102564102564, 0.7317073170731707]
precision_SVM: [0.9795918367346939, 1.0, 0.9344262295081968, 0.8805970149253731, 0.7894736842105263]
recall_SVM: [0.5393258426966292, 0.6179775280898876, 0.6404494382022472, 0.6629213483146067, 0.6818181818181818]
mean of Gmean_s: 0.6735388238771454
mean of AUC_s: 0.7815628192032686
mean of f1_s: 0.7415317272570718
mean of precision_s: 0.916817753075758
mean of recall_s: 0.6284984678243105
standard deviation of Gmean_s: 0.031343480305295074
standard deviation of AUC_s: 0.02384162526806158
standard deviation of f1_s: 0.028564588455855775
standard deviation of precision_s: 0.08470298691924626
standard deviation of recall_s: 0.055301897661086424


In [169]:
list=['pageblocks13vs4',stat.mean(Auc_k),stat.stdev(Auc_k),stat.mean(Gmean_k),stat.stdev(Gmean_k),stat.mean(Auc_d),stat.stdev(Auc_d),stat.mean(Gmean_d),stat.stdev(Gmean_d),stat.mean(Auc_s),stat.stdev(Auc_s),stat.mean(Gmean_s),stat.stdev(Gmean_s)]
list

df_excel = pd.read_excel('/content/drive/MyDrive/pso_1.xlsx')
df_excel

,Unnamed: 0.1,KNN,Unnamed: 2,Unnamed: 3,Unnamed: 4,Decision Tree,Unnamed: 6,Unnamed: 7,Unnamed: 8,SVM,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN
1,Dataset,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev
2,ecoli0137vs26,0.959495,0.040136,0.92449,0.072456,0.959798,0.027859,0.923228,0.052104,0.946633,0.061364,0.905228,0.105836
3,shuttle0vs4,0.999414,0.000802,0.998826,0.001607,1.0,0.0,1.0,0.0,0.998828,0.000655,0.997651,0.001313
4,yeast1vs7,0.953475,0.075796,0.923035,0.116982,0.938317,0.063328,0.89309,0.093762,0.963953,0.080602,0.94367,0.125957
5,shuttle2vs4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6,glass016vs2,0.925714,0.102718,0.885581,0.143202,0.885714,0.108327,0.827411,0.127491,0.591429,0.180871,0.723895,0.066047
7,glass016vs5,0.885714,0.137396,0.845013,0.164663,0.88,0.149352,0.84569,0.173688,0.951429,0.065153,0.914833,0.107251


In [170]:
df_excel.loc[len(df_excel)]=list
df_excel.to_excel('/content/drive/MyDrive/pso_1.xlsx')